In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import shutil

In [ ]:
# Carregar o modelo treinado
model = tf.keras.models.load_model(r"model/modelo_solda.h5")

# Diretório de novas imagens
input_dir = r"image/classificacao/novas"
train_dir = r"image/classificacao"  # Onde a IA vai mover as imagens classificadas
classes = ["boa", "ruim", "incertas"]

In [ ]:
# Função para garantir que as pastas existam
def verificar_pastas():
    for classe in classes:
        pasta = os.path.join(train_dir, classe)
        if not os.path.exists(pasta):
            os.makedirs(pasta)
            print(f"Pasta {classe} criada em {pasta}")
    incerta_pasta = os.path.join(train_dir, "incertas")
    if not os.path.exists(incerta_pasta):
        os.makedirs(incerta_pasta)
        print(f"Pasta 'incertas' criada em {incerta_pasta}")

In [ ]:
# Listar imagens para classificação
imagens = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

In [ ]:
# Função de processamento de imagem
def processar_imagem(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))  # Mesmo tamanho usado no treinamento
    img = img / 255.0  # Normalizar
    img = np.expand_dims(img, axis=0)  # Expandir dimensão para o modelo
    return img

In [ ]:
# Garantir que as pastas existam
verificar_pastas()

In [ ]:
# Classificar as imagens
for img_nome in imagens:
    img_path = os.path.join(input_dir, img_nome)
    img_processada = processar_imagem(img_path)

    # Fazer previsão
    predicao = model.predict(img_processada)[0]
    classe_predita = np.argmax(predicao)
    classe_nome = classes[classe_predita]

    print(f"Imagem {img_nome} classificada como: {classe_nome}")

    # Perguntar se a classificação está correta
    resposta = input(f"A classificação está correta? (s/n): ").strip().lower()

    if resposta == "n":
        nova_classe = input("Digite a classe correta (boa/ruim/incerta): ").strip().lower()
        if nova_classe in classes:
            destino = os.path.join(train_dir, nova_classe, img_nome)
            shutil.move(img_path, destino)
            print(f"Imagem movida para: {destino}")
        else:
            print("Classe inválida! A imagem não foi movida.")
    else:
        # Mover a imagem para a pasta correspondente à classificação
        destino = os.path.join(train_dir, classe_nome, img_nome)
        shutil.move(img_path, destino)
        print(f"Imagem movida para: {destino}")

print("Classificação finalizada!")

Classificação finalizada!
